In [1]:
import cv2
import json
import trt_pose.coco
import torch
import time
import numpy as np
import pyrealsense2 as rs
from torch2trt import TRTModule
from trt_pose.draw_objects import DrawObjects
from trt_pose.parse_objects import ParseObjects

OPTIMIZED_MODEL = 'pose_detection/resnet18_baseline_att_224x224_A_epoch_249_trt.pth'
HUMAN = 'pose_detection/human_pose.json'

In [2]:
def preprocess(image):
    mean = torch.Tensor([0.485, 0.456, 0.406]).to(device).view(1, 3, 1, 1)
    std = torch.Tensor([0.229, 0.224, 0.225]).to(device).view(1, 3, 1, 1)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image = cv2.resize(image, (224, 224))
    image = torch.from_numpy(image).permute(2, 0, 1).unsqueeze(0).float().to(device)
    image = image.div(255.0)
    image = (image - mean) / std
    return image

# Load model and topology
with open(HUMAN, 'r') as f:
    human_pose = json.load(f)
topology = trt_pose.coco.coco_category_to_topology(human_pose)

# Instantiate TensorRT
model_trt = TRTModule()
model_trt.load_state_dict(torch.load(OPTIMIZED_MODEL))

# Doing stuff (tbd)
parse_objects = ParseObjects(topology)
draw_objects = DrawObjects(topology)

device = torch.device('cuda')

class PIDController:
    def __init__(self, kp, ki, kd, set_point):
        self.kp = kp
        self.ki = ki
        self.kd = kd
        self.set_point = set_point
        self.last_error = 0.0
        self.integral = 0.0

    def update(self, current_value):
        error = self.set_point - current_value
        self.integral += error
        derivative = error - self.last_error
        self.last_error = error
        return self.kp * error + self.ki * self.integral + self.kd * derivative

In [3]:
from rover.controls import *
rover = Controls()

Initializing Servos
Initializing ServoKit
Done initializing


In [6]:
from pose_detection.pose_detection import *


WIDTH = 640
HEIGHT = 480
BRAKE_DISTANCE = 1269

# Initialize RealSense camera
pipeline = rs.pipeline()
config = rs.config()
config.enable_stream(rs.stream.color, WIDTH, HEIGHT, rs.format.bgr8, 30)
config.enable_stream(rs.stream.depth, 640, 480, rs.format.z16, 30)
pipeline.start(config)

body = Body()
speed = 0
pose_change = False

prev_pose = "No Pose"
center = [WIDTH/2, HEIGHT/2]
current_angle = 90
started = False

#DESIRED_DISTANCE = 1200
#pid = PIDController(kp=0.1, ki=0.01, kd=0.05, set_point=DESIRED_DISTANCE)

prev_frame_time = time.time()

try:
    while True:
        frames = pipeline.wait_for_frames()
        color_frame = frames.get_color_frame()
        depth_frame = frames.get_depth_frame()

        # Convert depth frame to numpy array
        depth_image = np.asanyarray(depth_frame.get_data())

        if not color_frame or not depth_frame:
            continue

        # Convert images to numpy arrays
        color_image = np.asanyarray(color_frame.get_data())

        # Process the image for pose estimation                # TODO: Think about speed being 1 and incrementing by 1
        data = preprocess(color_image)
        cmap, paf = model_trt(data)
        cmap, paf = cmap.detach().cpu(), paf.detach().cpu()
        counts, objects, peaks = parse_objects(cmap, paf)
        
        # Classify the pose and display it
        pose_classification, chest_in_frame = classify_pose(peaks, counts, objects, topology, body)
        cv2.putText(color_image, pose_classification, (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
        
        if pose_classification == "T-Pose":
            started = True
            speed = 1
            rover.forward(speed)
        elif pose_classification == "S-Pose":
            started = False
            rover.brake()
        
        if started == True:
            # Access depth data (in millimeters)
            depth_value_at_pixel = depth_image[int(body.chest[1]), int(body.chest[0])]
            print(f"Depth value at {body.chest[0]}, {body.chest[1]} pixel: {depth_value_at_pixel}")

            if depth_value_at_pixel < BRAKE_DISTANCE:
                rover.brake()
                speed = 1
            else:
                if chest_in_frame:
                    rover.forward(speed)

                x_diff = body.chest[0] - center[0]
                x_diff = (0.28125 * x_diff) + 90
                current_angle = x_diff
                print(current_angle)
                rover.turn(current_angle)
            
                prev_chest = body.chest

        
        # Draw poses and display them
        #draw_objects(color_image, 
        # 
        # counts, objects, peaks)
        draw_keypoints_and_values(color_image, counts, objects, peaks, topology)

        # Calculate FPS and display it
        current_frame_time = time.time()
        fps = (1 / (current_frame_time - prev_frame_time))
        prev_frame_time = current_frame_time
        fps_text = f'FPS: {fps:.0f}'
        cv2.putText(color_image, fps_text, (color_image.shape[1] - 150, color_image.shape[0] - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2)



        # Display the image
        cv2.imshow('Pose Estimation', color_image)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

finally:
    
    rover.brake()
    pipeline.stop()
    cv2.destroyAllWindows()

Speeding up to 36 

Depth value at 558, 218 pixel: 2981
Speeding up to 36 

156.9375
Speeding up to 36 

Depth value at 582, 237 pixel: 3163
Speeding up to 36 

163.6875
Depth value at 582, 237 pixel: 3131
163.6875
Depth value at 582, 237 pixel: 3423
163.6875
Depth value at 582, 237 pixel: 4153
163.6875
Depth value at 582, 237 pixel: 0
Braking 

Depth value at 582, 237 pixel: 3520
163.6875
Depth value at 582, 237 pixel: 3368
Speeding up to 36 

163.6875
Depth value at 582, 237 pixel: 3163
163.6875
Depth value at 582, 237 pixel: 3069
163.6875
Depth value at 582, 237 pixel: 2925
163.6875
Depth value at 582, 237 pixel: 2925
163.6875
Depth value at 582, 237 pixel: 2898
163.6875
Depth value at 582, 237 pixel: 2745
163.6875
Depth value at 582, 237 pixel: 2745
163.6875
Depth value at 582, 237 pixel: 2757
163.6875
Depth value at 582, 237 pixel: 2733
163.6875
Depth value at 582, 237 pixel: 2845
163.6875
Depth value at 582, 237 pixel: 2685
163.6875
Depth value at 582, 237 pixel: 2685
163.6875
De